<a href="https://colab.research.google.com/github/FNeffati/PDS/blob/main/pds_neural_net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import xarray as xr
# Open the datasets

!wget https://data.ngdc.noaa.gov/platforms/solar-space-observing-satellites/goes/goes16/l2/data/xrsf-l2-avg1m_science/sci_xrsf-l2-avg1m_g16_s20170207_e20231002_v2-2-0.nc
!wget https://data.ngdc.noaa.gov/platforms/solar-space-observing-satellites/goes/goes17/l2/data/xrsf-l2-avg1m_science/sci_xrsf-l2-avg1m_g17_s20180601_e20230110_v2-2-0.nc
!wget https://data.ngdc.noaa.gov/platforms/solar-space-observing-satellites/goes/goes18/l2/data/xrsf-l2-avg1m_science/sci_xrsf-l2-avg1m_g18_s20220902_e20231002_v2-2-0.nc
!wget https://data.ngdc.noaa.gov/platforms/solar-space-observing-satellites/goes/goes16/l2/data/xrsf-l2-flsum_science/sci_xrsf-l2-flsum_g16_s20170209_e20231002_v2-2-0.nc
!wget https://data.ngdc.noaa.gov/platforms/solar-space-observing-satellites/goes/goes17/l2/data/xrsf-l2-flloc_science/sci_xrsf-l2-flsum_g17_s20180601_e20230110_v2-2-0.nc
!wget https://data.ngdc.noaa.gov/platforms/solar-space-observing-satellites/goes/goes18/l2/data/xrsf-l2-flsum_science/sci_xrsf-l2-flsum_g18_s20220905_e20231002_v2-2-0.nc



--2023-10-05 18:50:55--  https://data.ngdc.noaa.gov/platforms/solar-space-observing-satellites/goes/goes16/l2/data/xrsf-l2-avg1m_science/sci_xrsf-l2-avg1m_g16_s20170207_e20231002_v2-2-0.nc
Resolving data.ngdc.noaa.gov (data.ngdc.noaa.gov)... 140.172.190.8
Connecting to data.ngdc.noaa.gov (data.ngdc.noaa.gov)|140.172.190.8|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 191626378 (183M) [application/x-netcdf]
Saving to: ‘sci_xrsf-l2-avg1m_g16_s20170207_e20231002_v2-2-0.nc.1’

sci_xrsf-l2-avg1m_g 100%[===================>] 182.75M  5.21MB/s    in 46s     

2023-10-05 18:51:42 (3.96 MB/s) - ‘sci_xrsf-l2-avg1m_g16_s20170207_e20231002_v2-2-0.nc.1’ saved [191626378/191626378]

--2023-10-05 18:51:42--  https://data.ngdc.noaa.gov/platforms/solar-space-observing-satellites/goes/goes17/l2/data/xrsf-l2-avg1m_science/sci_xrsf-l2-avg1m_g17_s20180601_e20230110_v2-2-0.nc
Resolving data.ngdc.noaa.gov (data.ngdc.noaa.gov)... 140.172.190.8
Connecting to data.ngdc.noaa.gov (data

In [9]:


file_path_16 = '/content/sci_xrsf-l2-avg1m_g16_s20170207_e20231002_v2-2-0.nc'
flare_summary_path_16 = '/content/sci_xrsf-l2-avg1m_g17_s20180601_e20230110_v2-2-0.nc'
file_path_17 = '/content/sci_xrsf-l2-avg1m_g18_s20220902_e20231002_v2-2-0.nc'
flare_summary_path_17 = '/content/sci_xrsf-l2-flsum_g17_s20180601_e20230110_v2-2-0.nc'
file_path_18 = '/content/sci_xrsf-l2-flsum_g18_s20220905_e20231002_v2-2-0.nc'
flare_summary_path_18 = '/content/sci_xrsf-l2-flsum_g18_s20220905_e20231002_v2-2-0.nc'

ds16 = xr.open_dataset(file_path_16)
ds16_flare = xr.open_dataset(flare_summary_path_16)

ds17 = xr.open_dataset(file_path_17)
ds17_flare = xr.open_dataset(flare_summary_path_17)

ds18 = xr.open_dataset(file_path_18)
ds18_flare = xr.open_dataset(flare_summary_path_18)



In [11]:
def process_telescope_data(file_path, flare_summary_path, telescope_number):
    ds = xr.open_dataset(file_path)
    ds2 = xr.open_dataset(flare_summary_path)

    # Convert xarray DataArrays to pandas DataFrames
    ds = ds[['xrsa_flux', 'xrsb_flux', 'time']].to_dataframe().reset_index()
    ds2 = ds2[['status', 'time']].to_dataframe().reset_index()

    # Add telescope identifier
    ds['telescope'] = telescope_number
    ds2['telescope'] = telescope_number

    return ds, ds2

# Process data for each telescope
ds_16, ds2_16 = process_telescope_data(file_path_16, flare_summary_path_16, 16)
ds_17, ds2_17 = process_telescope_data(file_path_17, flare_summary_path_17, 17)
ds_18, ds2_18 = process_telescope_data(file_path_18, flare_summary_path_18, 18)

# Concatenate data from all telescopes
ds = pd.concat([ds_16, ds_17, ds_18])
ds2 = pd.concat([ds2_16, ds2_17, ds2_18])



flare_statuses = ['EVENT_START', 'EVENT_PEAK', 'EVENT_END', 'POST_EVENT']
ds2['status'] = ds2['status'].apply(lambda x: 1 if x in flare_statuses else 0)

# Merge data
merged_data = pd.merge(ds, ds2, on=['time', 'telescope'], how='outer')

# Handle missing values
merged_data.fillna(0, inplace=True)

# Extract features and target
X = merged_data[['xrsa_flux', 'xrsb_flux']]
y = merged_data['status']

# Normalize features
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Apply SMOTE
smote = SMOTE(random_state=42)
X_train, y_train = smote.fit_resample(X_train, y_train)


KeyError: ignored

0.0    445295
1.0      8593
Name: status, dtype: int64


In [ ]:


# Building the model
model = keras.Sequential([
    keras.layers.Dense(128, activation='relu', input_shape=(2,)),  # 2 input features: 'xrsa_flux' and 'xrsb_flux'
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')  # Binary classification
])

# Compiling the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Training the model
model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print('\nTest accuracy:', test_acc)


In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, roc_auc_score

# Predict probabilities for the positive class
y_prob = model.predict(X_test)

# Compute ROC curve
fpr, tpr, thresholds = roc_curve(y_test, y_prob)

# Compute AUC
auc = roc_auc_score(y_test, y_prob)

plt.figure(figsize=(10, 7))
plt.plot(fpr, tpr, color='blue', label='ROC curve (AUC = %0.2f)' % auc)
plt.plot([0, 1], [0, 1], color='red', linestyle='--')  # Diagonal
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.grid(True)
plt.show()


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

y_pred = (model.predict(X_test) > 0.5).astype("int32")
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))


3546/3546 [==============================] - 7s 2ms/step
[[100960  10424]
 [   261   1827]]
              precision    recall  f1-score   support

         0.0       1.00      0.91      0.95    111384
         1.0       0.15      0.88      0.25      2088

    accuracy                           0.91    113472
   macro avg       0.57      0.89      0.60    113472
weighted avg       0.98      0.91      0.94    113472

